# Homework

**Due Date: 2024-06-27, 23:59**

You will need to succeed in this homework to be eligable for the final exam.

Should in the grey shaded panel not be a "ds24", then you will struggle with your imports -- and we will need to change the kernel: click on kernel/change kernel/ds24

You are allowed to work in teams of 3. Make sure to individually submit the homework, each containing the full names of your team members.


**CAUTION** There are a few training runs in this notebook, which you are of course allowed to change to your heart's content, just make sure that on submission, you change the epoch variable back to the original value. Otherwise the autograder will fail you due to runtime issues with your notebook!


In [ ]:
members = []

In [ ]:
import torch
import numpy as np
import random

# Task 1: Dataset
As a Machine Learner, you will often need to create custom datasets for your projects.
In this task, you will create a custom dataset class, ShapeDataset, which generates images with random placements of specific shapes. The shapes include horizontal lines, vertical lines, diagonal lines (including off-diagonals in both directions with the exception of the edges) , and 'L' shapes (the L should really only be 3 pixels). Each image will be a 9x9 grid where the shape is represented by 1s and the background is represented by 0s.  Should you not be able to write one of these, just continue by returning the empty image; where the image contains the randomly placed operation index (horizontal=0, vertical=1, diagonal=2, L=3).

In [ ]:
def horizontal_line():
    img = np.zeros((9, 9), dtype=int)
    # YOUR CODE HERE
    raise NotImplementedError
    return img


def vertical_line():
    img = np.zeros((9, 9), dtype=int)
    # YOUR CODE HERE
    raise NotImplementedError
    return img


def diagonal_line():
    img = np.zeros((9, 9), dtype=int)
    # YOUR CODE HERE
    raise NotImplementedError
    return img


def l_shape():
    img = np.zeros((9, 9), dtype=int)
    # YOUR CODE HERE
    raise NotImplementedError
    return img


Now we can write a dataset class, that returns us our images and labels using our helper functions.

In [ ]:
from torch.utils.data import Dataset


class ShapeDataset(Dataset):
    def __init__(self, length):
        # YOUR CODE HERE
        raise NotImplementedError

    def __len__(self):
        # YOUR CODE HERE
        raise NotImplementedError

    def __getitem__(self, idx):
        # YOUR CODE HERE
        raise NotImplementedError
        img = torch.tensor(img,
                           dtype=torch.float32)  # we make sure that it is a tensor that pytorch can use!
        return img, label


In [ ]:
# Let's check if your dataset works in a DataLoader:
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

dataset = ShapeDataset(
    10)  # notice, that the length is in our simulated case only relevant for how many "epochs" the data loader will run
dataloader = DataLoader(dataset, batch_size=3, shuffle=True)

for imgs, labels in dataloader:
    # Create the plot
    plt.imshow(imgs[0], cmap='viridis', interpolation='none')
    plt.colorbar()
    plt.title('2D Tensor Plot')
    plt.show()
    print(imgs.shape, labels)  # notice the batch dimension!
    break

Nice, now that we have a dataset, let us build a classifier capable of learning how to classify the base shapes

# Task 2: Convolution Operation
The task here is to write a simple convolution module with a single filter and calculate the forward pass manually. Make sure not to use any preimplemented `F.conv2d` operation or the nn.Conv2D. We will check that you didn't! Copying the src will be considered plagiarism and disqualify you. 
Of course, you should validate your solution against a F.conv2d from PyTorch to check if you are on the right track.  


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Define the SimpleConvolution module from previous example
class SimpleConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(SimpleConvolution, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Initialize the filter with random values
        # The Parameter is important to let PyTorch know that this tensor should be trained
        self.filter = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))

    def forward(self, x):
        # YOUR CODE HERE
        raise NotImplementedError
        return output


# Task 3: Simple Convolutional Neural Network
Alright, now that we have a dataset and a convolutional module, we can try and learn a simple classifier network and training loop, that 
will help us perfectly classifiy the shapes (in this task, should your own implementation not work, you are allowed to use pre-implemented versions, but how cool would it be if your own operations did the job -- but you probably will want to have more than one filter to learn).

You might find pooling operations helpful, but you can also try without them. 

**ATTENTION** you will need to comment the train call before submission otherwise this will result in a timeout during auto-grading and fail you! Don't make me manually grade you! 

In [ ]:
import torch
import torch.nn as nn


class SimpleCNN(nn.Module):
    def __init__(self, num_classes, kernel_size):
        super(SimpleCNN, self).__init__()
        # YOUR CODE HERE
        raise NotImplementedError

    def forward(self, x):
        # YOUR CODE HERE
        raise NotImplementedError

        return x

    def predict(self, x):
        # hint:
        # think about why this function is here and what output your forward should have
        # -- you won't use the predict during your training loop though!
        return torch.argmax(self.forward(x), dim=1)


In [ ]:
from torch import optim


def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        # YOUR CODE HERE
        raise NotImplementedError
        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}')


In [ ]:
model = SimpleCNN(4, 3)

train(
    # YOUR CODE HERE
    raise NotImplementedError
    num_epochs=100
)

Let's check if the model did something useful after your training (this part is not graded, but can help you find out what is wrong with your model).

In [ ]:
def evaluate_model_with_confusion_matrix(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.unsqueeze(1)  # Add channel dimension
            outputs = model.predict(images)
            y_pred.extend(outputs.numpy())
            y_true.extend(labels.numpy())

    model.train()
    return y_true, y_pred


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues',
                xticklabels=['Horizontal', 'Vertical', 'Diagonal', 'L-Shape'],
                yticklabels=['Horizontal', 'Vertical', 'Diagonal', 'L-Shape'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


# Use the modified evaluation function
test_dataset = ShapeDataset(200)  # Generate a separate dataset for testing
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
y_true, y_pred = evaluate_model_with_confusion_matrix(model, test_loader)

# Plot the confusion matrix
plot_confusion_matrix(y_true, y_pred)


Congratulations to your first successful Neural Network model!

# Task 4: A mysterious Network
Below you will find an intricate network design and you will have to answer some questions about it, so carefully read the code below and maybe consult the documentation of one or the other function. 

Hint: while the Conv2d strides with a kernel over the image to extract a feature map, the Conv2dTranspose does the opposite it takes a feature map, and takes a value from the feature map, multiplies it with each value of the kernel respectively and places them into its output tensor. Think of it as taking in a feature map and producing an image. 




The following plotting function is just a convenience function and can be ignored:) 

In [ ]:
import matplotlib.pyplot as plt


def plot_tensor_batch(tensor):
    # Ensure tensor is CPU tensor and convert to numpy
    tensor = tensor.cpu().numpy()

    # Determine the number of images
    num_images = tensor.shape[0]

    # Calculate the number of columns and rows to display
    num_cols = int(num_images ** 0.5)
    num_rows = (num_images + num_cols - 1) // num_cols

    # Set up the plot
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(2 * num_cols, 2 * num_rows))
    axes = axes.flatten()

    # Loop through all the images in the tensor
    for i, img in enumerate(tensor):
        # Handle grayscale or single channel data
        if img.shape[0] == 1:
            axes[i].imshow(img[0], cmap='gray', interpolation='none')
        else:
            # Convert from CHW to HWC format for plotting
            img = img.transpose(1, 2, 0)
            axes[i].imshow(img, interpolation='none')

        axes[i].axis('off')

    # Hide unused subplots
    for ax in axes[num_images:]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        self.layers = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(100, 128, 7, 1, 0, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # state size. 128 x 7 x 7
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # state size. 64 x 14 x 14
            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. 1 x 28 x 28
        )

    def forward(self, input):
        return self.layers(input)


class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.layers = nn.Sequential(
            # input is (1 x 28 x 28)
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 64 x 14 x 14
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 128 x 7 x 7
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.layers(input)

In [ ]:
# example usage of g
g = G()
vec = torch.randn(2, 100, 1, 1)
new_image = g(vec)

plot_tensor_batch(new_image.detach())



In [ ]:
# example usage of d
d = D()
img = torch.randn(2, 1, 28, 28)
output = d(img)
print(output)

Here we will just quickly collect the MNIST dataset. It comprises of 28x28 images of handwritten digits from 0 to 9.

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_data = datasets.MNIST(root='./data', train=True, download=False, transform=transform)
dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

In [ ]:
plot_tensor_batch(next(iter(dataloader))[0])

And here comes the intricate wiring of the two networks in their training loop:

In [ ]:
num_epochs = 0  # PLEASE MAKE SURE TO RESET THE VALUE TO ZERO BEFORE SUBMISSION or the autograding might fail you!
# but during this task, you may want to look what this training loop does for you. A value of 50 might already give you a descent idea in a reasonable time :) 

In [ ]:
# Initialize models
g = G()
d = D()

# Loss and optimizer
criterion = nn.BCELoss()
lr = 0.0002
optimizerG = optim.Adam(g.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerD = optim.Adam(d.parameters(), lr=lr, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        # Update d
        optimizerD.zero_grad()
        real_labels = torch.ones(real_images.size(0), 1)
        real_output = d(real_images)
        real_loss = criterion(real_output, real_labels)

        noise = torch.randn(real_images.size(0), 100, 1, 1)
        fake_images = g(noise)
        fake_labels = torch.zeros(real_images.size(0), 1)
        fake_output = d(fake_images.detach())
        fake_loss = criterion(fake_output, fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizerD.step()

        # Update g
        optimizerG.zero_grad()
        fake_output = d(fake_images)
        g_loss = criterion(fake_output, real_labels)  # Trick  d
        g_loss.backward()
        optimizerG.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss D: {d_loss.item()}, Loss G: {g_loss.item()}')



In [ ]:
torch.save(g.state_dict(), 'g.pth')
torch.save(d.state_dict(), 'd.pth')
# this way you can also load the model states back in later

In [ ]:
if False:
    g.load_state_dict(torch.load('g.pth'))
    d.load_state_dict(torch.load('d.pth'))


In [ ]:
# checking out new data from g
with torch.no_grad():
    test_noise = torch.randn(10, 100, 1, 1)
    new_images = g(test_noise)


In [ ]:
plot_tensor_batch(new_images)

Now please answer the following questions, giving all the right answer(s). (e.g. if A and B are correct: answer = set(["A", "B"]))

**What does the nn.Tanh() activation function in the generator's final layer do to the output images?**

A) Scales the output values to the range [0, 1]

B) Scales the output values to the range [-1, 1]

C) Applies a threshold to the output values

D) Normalizes the output values to have zero mean

In [ ]:
answer = None

**What is the purpose of detaching the fake images before passing them to the d during its update?**


A) To allow the g to receive gradients

B) To ensure the g's parameters are updated

C) To prevent the gradients from flowing back to the g during the d's update

D) To reduce the computational load



In [ ]:
answer = None

**Why do we update the d network with real and fake images separately?**

A) To ensure the g does not overfit

B) To reduce the training time

C) To provide the d with balanced examples of real and fake images

D) To increase the complexity of the model

In [ ]:
answer = None

**What is the ultimate goal of this training procedure?**

A) To classify images and distinguish between real and fake images

B) To generate realistic MNIST digit images

C) To reduce the size of the MNIST dataset

D) To improve the accuracy of the MNIST d network



In [ ]:
answer = None

**Why is it important for the g to "trick" the d?**

A) To ensure the d performs well on real images

B) To make the training process faster and more efficient

C) To make the d more robust against adversarial attacks

D) To improve the quality of the new images so they become indistinguishable from real images


In [ ]:
answer = None

**Which of the following use cases could be application cases of a modified vesion of the architecture?**

A) Creating realistic photographs of people who don't exist

B) Predicting house prices based on historical sales data

C) Image-to-image translation, such as turning a photograph into a painting

D) Classifying emails as spam or not spam

E) Generating high-resolution images from low-resolution inputs (super-resolution)

F) Clustering customer data for market segmentation
    
G) Text-to-image synthesis, such as generating images from textual descriptions

H) Generating synthetic medical images for training purposes

I) Predicting stock market trends based on past data

J) Voice conversion to make one person's voice sound like another's

In [ ]:
answer = None

# Task 5: Classification metrics
Take a look at the following confusion matrix. And compute the following metrics:

![Confusion](confusion.jpg)


In [ ]:
true_positives = None
false_positives_A = None  # false positives for class A
accuracy = None # make sure this is a devision e.g. 3/4 
precision_B = None  # precision for class B # same here!

# Task 6: Regression Tree
 The regression tree in Figure (\ref{fig:regression_tree}) is a model of a particular $min\_samples\_leaf$ value applied to the sklearn exemplary Diabetes dataset. This dataset has 442 observations with the following 10 feature columns: age, sex, body mass index (bmi), average blood pressure (bp) and six blood serum measurements (s1-s6). The target variable is a quantitative measure of disease progression one year after baseline. The statisticians were asked to construct a model that predicted target $y$ from covariates. Two hopes were evident here that the model would produce accurate baseline predictions of response for future patients and also that the form of the model would suggest which covariates were important factors in disease progression.
 
![Tree](regressiontree.jpg)
 


Specify, in which leaf node, the observation (age=41, sex=1, bmi=0.05, bp=0.006, s1=0.02, s2=0.01,
s3=-0.05, s4=-0.33, s5=0.006, s6=0.03) falls (leaf count starts at zero from the leftmost), and report
its predicted disease progression value one year after the baseline.

In [ ]:
node = None
value = None

Determine, given the plot, what value min_samples_leaf was used at most. 

In [ ]:
min_samples_leaf = None

**What is the relevance of the min_samples_leaf parameter to the Regression Tree’s model performance?**

A. It prevents overfitting by ensuring leaf nodes have a minimum number of samples, which helps the model generalize better.

B. It controls the maximum depth of the tree.

C. It sets the maximum number of leaf nodes in the tree.

D. It prevents overfitting by stopping the tree from growing too deep.

E. It is a hyperparameter.

In [ ]:
answer = None